# letter-recognition

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

# Import necessary modules
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

import pandasql as psql

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
#from itertools import cycle

from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Ignore harmless warnings

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the 'letter recognition' dataset

lettersdata = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\51 season Multinomial Classifier 02-Aug-2021\letter-recognition.csv", header=0)
lettersdata.head()

# Display the dataset information

lettersdata.info()

# Get the sub-set of data for letters A, B, P & R

lettersdata_N = psql.sqldf("select * \
from lettersdata \
where letter in ('A', 'B', 'C', 'D', 'E', \
'P', 'Q', 'R', 'S', 'T' )")

lettersdata_N.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
2,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
3,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
4,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7


In [3]:

# Display the new 'letter_recognition' data information

lettersdata_N.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7752 entries, 0 to 7751
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  7752 non-null   object
 1   xbox    7752 non-null   int64 
 2   ybox    7752 non-null   int64 
 3   width   7752 non-null   int64 
 4   height  7752 non-null   int64 
 5   onpix   7752 non-null   int64 
 6   xbar    7752 non-null   int64 
 7   ybar    7752 non-null   int64 
 8   x2bar   7752 non-null   int64 
 9   y2bar   7752 non-null   int64 
 10  xybar   7752 non-null   int64 
 11  x2ybar  7752 non-null   int64 
 12  xy2bar  7752 non-null   int64 
 13  xedge   7752 non-null   int64 
 14  xedgey  7752 non-null   int64 
 15  yedge   7752 non-null   int64 
 16  yedgex  7752 non-null   int64 
dtypes: int64(16), object(1)
memory usage: 1.0+ MB


In [4]:
# Display the count of each letter

lettersdata_N['letter'].value_counts(sort=False)

D    805
C    736
B    766
E    768
S    748
A    789
T    796
R    758
Q    783
P    803
Name: letter, dtype: int64

In [5]:
# Identify the dependent and Target variables

IndepVar = []
for col in lettersdata_N.columns:
    if col != 'letter':
        IndepVar.append(col)

TargetVar = 'letter'

x = lettersdata_N[IndepVar]
y = lettersdata_N[TargetVar]

In [6]:
# Import LabelEncoder and convert the target variable from object to integer values

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

#creating labelEncoder

le = preprocessing.LabelEncoder()

# Converting string labels into numbers.

y=le.fit_transform(y)

y=pd.DataFrame(y)

In [7]:
# Split into train and test sets

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 6)

# Copy the test data to back-up file

x_test_F1 = x_test.copy()

In [8]:
# Scaling the features

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled)

# Bagging Classifier

In [9]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model.
# We set'None' as a base estimator and set 100 to the number of estimators, then train the model
# with train data.

from sklearn.ensemble import BaggingClassifier

modelBAG = BaggingClassifier(base_estimator=None,
n_estimators=100, # number of LR models to create
max_samples=1.0, # each model is trained from randomly sampled 1 instance
max_features=1.0,
bootstrap=True, # set to False to use Pasting instead of Bagging
bootstrap_features=False,
oob_score=False,
warm_start=False,
n_jobs=None,
random_state=None,
verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data

y1_pred = modelBAG.predict(x_test)
y1_pred_proba = modelBAG.predict_proba(x_test)

In [10]:
# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y1_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y1_pred

# Class = Label 0-9

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);

    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);

    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy)); # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[173   0   0   0   0   0   0   0   0   0]
 [  1 145   0   1  16   3  10  12   8   2]
 [  1   0 166   0   1   0   1   0   0   1]
 [  2  15   0 138   0   1   4   6   3   3]
 [  0   2   1   0 162   0   1   2   0   0]
 [  0  28   0  56   0 201   3   5   2   0]
 [  0   0   2   0   1   0 184   0   1   1]
 [  3   2   0   1   0   0   0 155   0   0]
 [  2   6   0   0  19   0   3   0 165   0]
 [  0   0   1   0   1   0   0   0   1 214]]
Print Class: 0
TP=173, FP=0, TN=1756, FN=9
Accuracy: 0.995
Precision: 1.0
Sensitivity: 0.951
F1-Score: 0.975
Specificity: 1.0
Balanced Accuracy: 0.976
MCC: 0.972

Print Class: 1
TP=145, FP=53, TN=1687, FN=53
Accuracy: 0.945
Precision: 0.732
Sensitivity: 0.732
F1-Score: 0.732
Specificity: 0.97
Balanced Accuracy: 0.851
MCC: 0.702

Print Class: 2
TP=166, FP=4, TN=1764, FN=4
Accuracy: 0.996
Precision: 0.976
Sensitivity: 0.976
F1-Score: 0.976
Specificity: 0.998
Balanced Accuracy: 0.987
MCC: 0.974

Print Class: 3
TP=138, FP=34, TN=1708, FN=58
Accuracy: 0.953
Precision:

In [11]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean()

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 97.57%
Precision: 89.24%
Recall or Sensitivity: 87.92%
F1-Score: 0.8815
Specificity or True Nagative Rate: 98.66%
Balanced Accuracy: 93.3%

MCC: 0.8708



# Bagging Classifier with decision tree

In [23]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model.
# We set'Decision Tree' as a base estimator and set 100 to the number of estimators, then train the model
# with train data.

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

modelDT = DecisionTreeClassifier(criterion="entropy")

modelBAG = BaggingClassifier(base_estimator=modelDT,
n_estimators=100, # number of LR models to create
max_samples=1.0, # each model is trained from randomly sampled 1 instance
max_features=1.0,
bootstrap=True, # set to False to use Pasting instead of Bagging
bootstrap_features=False,
oob_score=False,
warm_start=False,
n_jobs=None,
random_state=None,
verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data

y2_pred = modelBAG.predict(x_test)
y2_pred_proba = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y2_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y2_pred

# Class = Label 0-9

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);

    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);

    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy)); # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[162   1   0   0   1   0   1   0   0   0]
 [  2 130   0   2  14   2  20  23  14   5]
 [  1   0 167   0   4   0   2   0   0   1]
 [  7  16   0 142   1   1  10   7   3   3]
 [  0   1   1   0 157   1   1   1   0   0]
 [  3  36   0  50   0 201   1   5   7   0]
 [  0   1   1   0   2   0 165   0   0   0]
 [  4   4   0   1   0   0   3 144   1   1]
 [  2   9   0   0  20   0   2   0 152   2]
 [  1   0   1   1   1   0   1   0   3 209]]
Print Class: 0
TP=162, FP=3, TN=1753, FN=20
Accuracy: 0.988
Precision: 0.982
Sensitivity: 0.89
F1-Score: 0.934
Specificity: 0.998
Balanced Accuracy: 0.944
MCC: 0.929

Print Class: 1
TP=130, FP=82, TN=1658, FN=68
Accuracy: 0.923
Precision: 0.613
Sensitivity: 0.657
F1-Score: 0.634
Specificity: 0.953
Balanced Accuracy: 0.805
MCC: 0.591

Print Class: 2
TP=167, FP=8, TN=1760, FN=3
Accuracy: 0.994
Precision: 0.954
Sensitivity: 0.982
F1-Score: 0.968
Specificity: 0.995
Balanced Accuracy: 0.988
MCC: 0.965

Print Class: 3
TP=142, FP=48, TN=1694, FN=54
Accuracy: 0.947
Preci

In [24]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean()

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 96.8%
Precision: 85.91%
Recall or Sensitivity: 84.09%
F1-Score: 0.8444
Specificity or True Nagative Rate: 98.21%
Balanced Accuracy: 91.14%

MCC: 0.8304



# Logistic Regression

In [26]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model.
# We set'Logistic Regression' as a base estimator and set 100 to the number of estimators, then train the model
# with train data.

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

modelLR = LogisticRegression()

modelBAG = BaggingClassifier(base_estimator=modelLR,
n_estimators=100, # number of LR models to create
max_samples=1.0, # each model is trained from randomly sampled 1 instance
max_features=1.0,
bootstrap=True, # set to False to use Pasting instead of Bagging
bootstrap_features=False,
oob_score=False,
warm_start=False,
n_jobs=None,
random_state=None,
verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data

y3_pred = modelBAG.predict(x_test)
y3_pred_proba = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y3_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y3_pred

# Class = Label 0-9

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);

    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);

    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy)); # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[162   0   0   0   0   0   3   1   2   0]
 [  1 186   0  17  17   5  16  24  29   5]
 [  2   0 158   0   2   0   0   0   1   2]
 [  1   3   0 168   0   2   4   8   1   3]
 [  0   0   6   0 145   1   3   0   0  10]
 [  1   2   1   3   0 189   0   0   6   3]
 [  2   2   1   3  10   1 170   1   5   3]
 [  3   3   0   1   3   3   4 144   3   4]
 [  8   2   0   3  10   0   6   0 127   5]
 [  2   0   4   1  13   4   0   2   6 186]]
Print Class: 0
TP=162, FP=6, TN=1750, FN=20
Accuracy: 0.987
Precision: 0.964
Sensitivity: 0.89
F1-Score: 0.926
Specificity: 0.997
Balanced Accuracy: 0.944
MCC: 0.919

Print Class: 1
TP=186, FP=114, TN=1626, FN=12
Accuracy: 0.935
Precision: 0.62
Sensitivity: 0.939
F1-Score: 0.747
Specificity: 0.934
Balanced Accuracy: 0.936
MCC: 0.732

Print Class: 2
TP=158, FP=7, TN=1761, FN=12
Accuracy: 0.99
Precision: 0.958
Sensitivity: 0.929
F1-Score: 0.943
Specificity: 0.996
Balanced Accuracy: 0.962
MCC: 0.938

Print Class: 3
TP=168, FP=22, TN=1720, FN=28
Accuracy: 0.974
Preci

In [28]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean()

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 96.86%
Precision: 85.85%
Recall or Sensitivity: 84.35%
F1-Score: 0.8465
Specificity or True Nagative Rate: 98.25%
Balanced Accuracy: 91.28%

MCC: 0.832



# KNN

In [31]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model.
# We set'KNeighborsClassifier' as a base estimator and set 100 to the number of estimators, then train the model
# with train data.

from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

modelKNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30,
p=2, metric='minkowski', metric_params=None, n_jobs=None)

modelBAG = BaggingClassifier(base_estimator=modelKNN,
n_estimators=100, # number of LR models to create
max_samples=1.0, # each model is trained from randomly sampled 1 instance
max_features=1.0,
bootstrap=True, # set to False to use Pasting instead of Bagging
bootstrap_features=False,
oob_score=False,
warm_start=False,
n_jobs=None,
random_state=None,
verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data

y4_pred = modelBAG.predict(x_test)
y4_pred_proba = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y4_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y4_pred

# Class = Label 0-9

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);

    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);

    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy)); # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[180   0   0   0   0   0   0   0   1   0]
 [  0 190   0   6   8   2   0  11   3   3]
 [  0   0 170   0   4   0   0   0   0   1]
 [  1   2   0 187   0   0   0   0   1   2]
 [  0   1   0   0 186   1   0   0   0   0]
 [  0   0   0   0   0 202   1   0   0   0]
 [  0   0   0   0   0   0 204   0   1   0]
 [  1   3   0   3   0   0   1 169   0   2]
 [  0   2   0   0   2   0   0   0 173   0]
 [  0   0   0   0   0   0   0   0   1 213]]
Print Class: 0
TP=180, FP=1, TN=1755, FN=2
Accuracy: 0.998
Precision: 0.994
Sensitivity: 0.989
F1-Score: 0.992
Specificity: 0.999
Balanced Accuracy: 0.994
MCC: 0.991

Print Class: 1
TP=190, FP=33, TN=1707, FN=8
Accuracy: 0.979
Precision: 0.852
Sensitivity: 0.96
F1-Score: 0.903
Specificity: 0.981
Balanced Accuracy: 0.97
MCC: 0.893

Print Class: 2
TP=170, FP=5, TN=1763, FN=0
Accuracy: 0.997
Precision: 0.971
Sensitivity: 1.0
F1-Score: 0.986
Specificity: 0.997
Balanced Accuracy: 0.998
MCC: 0.984

Print Class: 3
TP=187, FP=6, TN=1736, FN=9
Accuracy: 0.992
Precision: 0

In [32]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean()

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 99.32%
Precision: 96.81%
Recall or Sensitivity: 96.72%
F1-Score: 0.9674
Specificity or True Nagative Rate: 99.62%
Balanced Accuracy: 98.16%

MCC: 0.964



# Random Forest

In [29]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model.
# We set'RandomForestClassifier' as a base estimator and set 100 to the number of estimators, then train the model
# with train data.

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(criterion='gini', n_estimators=100, random_state=0)

modelBAG = BaggingClassifier(base_estimator=modelRF,
n_estimators=100, # number of LR models to create
max_samples=1.0, # each model is trained from randomly sampled 1 instance
max_features=1.0,
bootstrap=True, # set to False to use Pasting instead of Bagging
bootstrap_features=False,
oob_score=False,
warm_start=False,
n_jobs=None,
random_state=None,
verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data

y5_pred = modelBAG.predict(x_test)
y5_pred_proba = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y5_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y5_pred

# Class = Label 0-9

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);

    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);

    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy)); # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[174   0   0   0   0   0   0   0   0   0]
 [  0 182   0   0  22   2   8   8   5   3]
 [  1   0 169   0   4   0   0   0   0   0]
 [  0   2   0 190   0   0   3   4   1   2]
 [  0   1   0   0 157   0   0   0   0   0]
 [  1   9   0   6   0 203   2   1   2   0]
 [  0   1   1   0   3   0 193   0   0   0]
 [  1   2   0   0   0   0   0 167   0   1]
 [  5   1   0   0  14   0   0   0 172   0]
 [  0   0   0   0   0   0   0   0   0 215]]
Print Class: 0
TP=174, FP=0, TN=1756, FN=8
Accuracy: 0.996
Precision: 1.0
Sensitivity: 0.956
F1-Score: 0.978
Specificity: 1.0
Balanced Accuracy: 0.978
MCC: 0.976

Print Class: 1
TP=182, FP=48, TN=1692, FN=16
Accuracy: 0.967
Precision: 0.791
Sensitivity: 0.919
F1-Score: 0.85
Specificity: 0.972
Balanced Accuracy: 0.946
MCC: 0.835

Print Class: 2
TP=169, FP=5, TN=1763, FN=1
Accuracy: 0.997
Precision: 0.971
Sensitivity: 0.994
F1-Score: 0.983
Specificity: 0.997
Balanced Accuracy: 0.996
MCC: 0.981

Print Class: 3
TP=190, FP=12, TN=1730, FN=6
Accuracy: 0.991
Precision: 

In [30]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean()

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 98.81%
Precision: 94.51%
Recall or Sensitivity: 94.07%
F1-Score: 0.9407
Specificity or True Nagative Rate: 99.33%
Balanced Accuracy: 96.7%

MCC: 0.9354



# SVM

In [34]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model.
# We set'SVM' as a base estimator and set 100 to the number of estimators, then train the model
# with train data.

from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

modelSVMGaussian = SVC(kernel='rbf', random_state = None, class_weight=None,probability=True)

modelBAG = BaggingClassifier(base_estimator=modelSVMGaussian,
n_estimators=100, # number of LR models to create
max_samples=1.0, # each model is trained from randomly sampled 1 instance
max_features=1.0,
bootstrap=True, # set to False to use Pasting instead of Bagging
bootstrap_features=False,
oob_score=False,
warm_start=False,
n_jobs=None,
random_state=None,
verbose=0)

modelBAG.fit(x_train,y_train)

# Predict model with test data

y6_pred = modelBAG.predict(x_test)
y6_pred_proba = modelBAG.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y6_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y6_pred

# Class = Label 0-9

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);

    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);

    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);

    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy)); # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[180   0   0   0   0   0   0   0   0   0]
 [  0 196   0   4   4   1   4  16   2   3]
 [  1   0 169   0   4   0   0   0   0   0]
 [  0   1   0 192   0   0   0   3   1   2]
 [  0   1   1   0 181   0   0   0   0   1]
 [  1   0   0   0   1 202   1   0   0   0]
 [  0   0   0   0   1   0 201   0   0   0]
 [  0   0   0   0   2   2   0 161   0   2]
 [  0   0   0   0   5   0   0   0 174   0]
 [  0   0   0   0   2   0   0   0   3 213]]
Print Class: 0
TP=180, FP=0, TN=1756, FN=2
Accuracy: 0.999
Precision: 1.0
Sensitivity: 0.989
F1-Score: 0.994
Specificity: 1.0
Balanced Accuracy: 0.994
MCC: 0.994

Print Class: 1
TP=196, FP=34, TN=1706, FN=2
Accuracy: 0.981
Precision: 0.852
Sensitivity: 0.99
F1-Score: 0.916
Specificity: 0.98
Balanced Accuracy: 0.985
MCC: 0.909

Print Class: 2
TP=169, FP=5, TN=1763, FN=1
Accuracy: 0.997
Precision: 0.971
Sensitivity: 0.994
F1-Score: 0.983
Specificity: 0.997
Balanced Accuracy: 0.996
MCC: 0.981

Print Class: 3
TP=192, FP=7, TN=1735, FN=4
Accuracy: 0.994
Precision: 0.9

In [35]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean()

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 99.28%
Precision: 96.65%
Recall or Sensitivity: 96.44%
F1-Score: 0.9645
Specificity or True Nagative Rate: 99.59%
Balanced Accuracy: 98.03%

MCC: 0.9613

